In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
credit_data = pd.read_csv('credit_defaulters_data_with_fail_cases.csv')

In [3]:
credit_data.describe()

,age,profession,city,state,avg_income_per_annum,risk_flag,incorrect_ones
count,211385.000000,211385.000000,211385.000000,211385.000000,211385.000000,211385.000000,211385.000000
mean,0.520233,0.488545,0.499208,0.468311,0.027180,0.146633,0.120359
std,0.290568,0.499870,0.500001,0.498996,0.045728,0.353740,0.325381
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.275862,0.000000,0.000000,0.000000,0.009094,0.000000,0.000000
50%,0.534483,0.000000,0.000000,0.000000,0.018391,0.000000,0.000000
75%,0.775862,1.000000,1.000000,1.000000,0.032165,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
risk_labels = credit_data.pop('risk_flag')
data_cols = credit_data.columns
incorrect_labels = credit_data.pop('incorrect_ones')

In [5]:
credit_data

,age,profession,city,state,avg_income_per_annum
0,0.034483,0,1,1,0.040869
1,0.327586,1,1,0,0.040956
2,0.775862,1,1,1,0.032700
3,0.637931,0,0,0,0.010523
4,0.206897,0,0,0,0.055917
...,...,...,...,...,...
211380,0.241379,0,1,1,0.005113
211381,0.724138,0,1,1,0.002389
211382,0.448276,1,0,0,0.041340
211383,0.534483,0,0,1,0.066190


In [6]:
risk_labels

0         0
1         0
2         0
3         0
4         0
         ..
211380    1
211381    1
211382    1
211383    1
211384    1
Name: risk_flag, Length: 211385, dtype: int64

In [7]:
def data_splicing(train, labels, tr_cols, target):
    # Firstly, make d0 and d1 as risk_flag specific datasets
    d = pd.DataFrame(np.hstack((train.values, labels.values.reshape(-1, 1))), columns=tr_cols)
    d0 = d[d[target]==0]
    d1 = d[d[target]==1]

    # Randomized Data Splicing
    tr_0, ts_0 = train_test_split(d0.values, test_size=2000, random_state=None)
    tr_1, ts_1 = train_test_split(d1.values, test_size=2000, random_state=None)
    # # Validation Data
    # tr_0, trv_0 = train_test_split(tr_0, test_size=1000, random_state=None)
    # tr_1, trv_1 = train_test_split(tr_1, test_size=1000, random_state=None)

    print("Train 0, 1:", tr_0.shape, tr_1.shape)

    X_tr = np.concatenate((tr_0[:, :tr_0.shape[1]-1], tr_1[:, :tr_1.shape[1]-1]), axis=0)
    print(X_tr)
    y_tr = np.concatenate((tr_0[:, tr_0.shape[1]-1:], tr_1[:, tr_1.shape[1]-1:]), axis=0)
    y_tr = y_tr.ravel()
    print(y_tr.shape)

    X_ts = np.concatenate((ts_0[:, :ts_0.shape[1]-1], ts_1[:, :ts_1.shape[1]-1]), axis=0)
    print(X_ts)
    y_ts = np.concatenate((ts_0[:, ts_0.shape[1]-1:], ts_1[:, ts_1.shape[1]-1:]), axis=0)
    y_ts = y_ts.ravel()
    print(y_ts.shape)
    
    return tr_0, tr_1, X_ts, y_ts
    # X_trv = np.concatenate((trv_0[:, :trv_0.shape[1]-1], trv_1[:, :trv_1.shape[1]-1]), axis=0)
    # print(X_trv)
    # y_trv = np.concatenate((trv_0[:, trv_0.shape[1]-1:], trv_1[:, trv_1.shape[1]-1:]), axis=0)
    # y_trv = y_trv.ravel()
    # print(y_trv)

In [8]:
tr_0, tr_1, X_ts, y_ts = data_splicing(credit_data, incorrect_labels, data_cols, 'incorrect_ones')

Train 0, 1: (183943, 6) (23442, 6)
[[0.96551724 1.         0.         1.         0.00430533]
 [0.93103448 1.         1.         1.         0.01517873]
 [0.9137931  0.         1.         0.         0.0083069 ]
 ...
 [0.48275862 0.         1.         0.         0.01219749]
 [0.81034483 1.         1.         0.         0.01079366]
 [0.37931034 1.         0.         0.         0.04748495]]
(207385,)
[[0.74137931 0.         1.         1.         0.05155481]
 [0.10344828 1.         1.         0.         0.06132666]
 [0.96551724 0.         0.         0.         0.01422149]
 ...
 [0.81034483 1.         1.         0.         0.03561734]
 [0.0862069  1.         1.         0.         0.0102119 ]
 [0.60344828 1.         0.         0.         0.01659259]]
(4000,)


In [9]:
def rf_model_team_trainer(tr_0, tr_1, thresh=0.05):
    
    rf_models = []

    for i in range(3):

    #     # Randomized Data Splicing
    #     tr_0, ts_0 = train_test_split(d0.values, test_size=1000, random_state=None)
    #     tr_1, ts_1 = train_test_split(d1.values, test_size=1000, random_state=None)
        # Validation Data
        tr_0n, trv_0n = train_test_split(tr_0, test_size=2000, random_state=None)
        tr_1n, trv_1n = train_test_split(tr_1, test_size=2000, random_state=None)

    #     print("Train 0, 1:", tr_0.shape, tr_1.shape)

        X_tr = np.concatenate((tr_0n[:, :tr_0n.shape[1]-1], tr_1n[:, :tr_1n.shape[1]-1]), axis=0)
    #     print(X_tr)
        y_tr = np.concatenate((tr_0n[:, tr_0n.shape[1]-1:], tr_1n[:, tr_1n.shape[1]-1:]), axis=0)
        y_tr = y_tr.ravel()
    #     print(y_tr.shape)

    #     X_ts = np.concatenate((ts_0[:, :ts_0.shape[1]-1], ts_1[:, :ts_1.shape[1]-1]), axis=0)
    #     print(X_ts)
    #     y_ts = np.concatenate((ts_0[:, ts_0.shape[1]-1:], ts_1[:, ts_1.shape[1]-1:]), axis=0)
    #     y_ts = y_ts.ravel()
    #     print(y_ts.shape)

        X_trv = np.concatenate((trv_0n[:, :trv_0n.shape[1]-1], trv_1n[:, :trv_1n.shape[1]-1]), axis=0)
    #     print(X_trv)
        y_trv = np.concatenate((trv_0n[:, trv_0n.shape[1]-1:], trv_1n[:, trv_1n.shape[1]-1:]), axis=0)
        y_trv = y_trv.ravel()
    #     print(y_trv)

#         # define the undersampling method
#         undersample = RandomUnderSampler(random_state=None)
#     #     tl = TomekLinks()
#         # transform the dataset
#         X_new, y_new = undersample.fit_resample(X_tr, y_tr)
#     #     X_new, y_new = tl.fit_resample(X_new, y_new)

#     #     print(X_new[0, 0])

        rf = RandomForestRegressor(n_jobs=-1, random_state=0)
        rf.fit(X_tr, y_tr)
        
        def tes(a):
            if a>thresh:
                return 1
            return 0
        def rfpre(a):
            p = rf.predict(a)
            p=list(map(tes,p))
            return p

        ac = accuracy_score(y_trv, rfpre(X_trv))
        print(f"Accuracy: {ac}")
        if(ac>=0.86):
            rf_models.append(rf)
            print("Model", i+1, "--> Done")
        else:
            print("Less Accurate Model")
            
    return rf_models
            

In [10]:
rf_verf_models = rf_model_team_trainer(tr_0, tr_1)

Accuracy: 0.883
Model 1 --> Done
Accuracy: 0.89025
Model 2 --> Done
Accuracy: 0.9085
Model 3 --> Done


In [11]:
def rf_team_predictions(rf_models, test_x):
#     predicted_flags=np.zeros(test_x.shape[0]
    predicted_flags=[]
    
    def tes(a):
        if a>0.05:
            return 1
        return 0
        
    def rfpre(a, rf):
        p = rf.predict(a)
        p=list(map(tes,p))
        return p
    
    for model in rf_models:
        predicted_flags.append(np.array(rfpre(test_x, model)).astype('int64'))

#     predicted_flags/=len(rf_models)
    predicted_flags = np.array(predicted_flags)
    final_prediction = np.zeros(predicted_flags.shape[1])
    for i in range(predicted_flags.shape[1]):
        final_prediction[i] = np.bincount(predicted_flags[:, i].ravel()).argmax()
    
    return final_prediction.astype('int64')

In [12]:
predicted_flags = rf_team_predictions(rf_verf_models, X_ts)
print(classification_report(y_ts, predicted_flags))
print(accuracy_score(y_ts, predicted_flags))
print(np.unique(predicted_flags))

print(confusion_matrix(y_ts, predicted_flags))

              precision    recall  f1-score   support

         0.0       0.95      0.82      0.88      2000
         1.0       0.84      0.95      0.89      2000

    accuracy                           0.89      4000
   macro avg       0.89      0.89      0.89      4000
weighted avg       0.89      0.89      0.89      4000

0.88775
[0 1]
[[1642  358]
 [  91 1909]]


In [13]:
predicted_flags = rf_team_predictions(rf_verf_models, credit_data.values)
print(classification_report(incorrect_labels.values.ravel(), predicted_flags))
print(accuracy_score(incorrect_labels.values.ravel(), predicted_flags))
print(np.unique(predicted_flags))

              precision    recall  f1-score   support

           0       1.00      0.84      0.91    185943
           1       0.46      1.00      0.63     25442

    accuracy                           0.86    211385
   macro avg       0.73      0.92      0.77    211385
weighted avg       0.93      0.86      0.88    211385

0.860765901080966
[0 1]


In [14]:
print(confusion_matrix(incorrect_labels.values.ravel(), predicted_flags))

[[156621  29322]
 [   110  25332]]


In [15]:
credit_data['incorrect_preds'] = predicted_flags
credit_data['risk_flag'] = risk_labels.values
credit_data

,age,profession,city,state,avg_income_per_annum,incorrect_preds,risk_flag
0,0.034483,0,1,1,0.040869,0,0
1,0.327586,1,1,0,0.040956,0,0
2,0.775862,1,1,1,0.032700,0,0
3,0.637931,0,0,0,0.010523,0,0
4,0.206897,0,0,0,0.055917,0,0
...,...,...,...,...,...,...,...
211380,0.241379,0,1,1,0.005113,1,1
211381,0.724138,0,1,1,0.002389,1,1
211382,0.448276,1,0,0,0.041340,1,1
211383,0.534483,0,0,1,0.066190,0,1


In [16]:
# Split the data based on 0s and 1s in incorrect_preds
def split_up_via_incorrect(credit_data):
    return credit_data.loc[(credit_data.incorrect_preds == 0) | (credit_data.risk_flag == 1)].index,\
    credit_data.loc[(credit_data.incorrect_preds == 1) | (credit_data.risk_flag == 1)].index

In [17]:
zero_i, one_i = split_up_via_incorrect(credit_data)

In [18]:
print(zero_i, one_i)

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 9,     10,
            ...
            211375, 211376, 211377, 211378, 211379, 211380, 211381, 211382,
            211383, 211384],
           dtype='int64', length=179980) Int64Index([     8,     50,     56,     60,     70,    106,    190,    218,
               254,    303,
            ...
            211375, 211376, 211377, 211378, 211379, 211380, 211381, 211382,
            211383, 211384],
           dtype='int64', length=62401)


In [19]:
credit_data.drop(['incorrect_preds'], axis=1, inplace=True)
credit_data.drop(['risk_flag'], axis=1, inplace=True)

In [20]:
def final_model_training_and_preds(credit_data, risk_labels, zero_i, one_i):
    
    data_cols = np.array(list(credit_data.columns) + ['risk_flag'])
    rf_model_team_preds = []
    
#     for data_inds in [zero_i, one_i]:
    tr_0, tr_1, X_ts, y_ts = data_splicing(credit_data.iloc[zero_i], risk_labels.iloc[zero_i], data_cols, 'risk_flag')
    rf_models = rf_model_team_trainer(tr_0, tr_1)
    predicted_flags = rf_team_predictions(rf_models, X_ts)
    print(classification_report(y_ts, predicted_flags))
    print(accuracy_score(y_ts, predicted_flags))
    print(np.unique(predicted_flags))

    print(confusion_matrix(y_ts, predicted_flags))
    
    predicted_flags = rf_team_predictions(rf_models, credit_data.iloc[zero_i].values)
    rf_model_team_preds.append(predicted_flags)
    
    tr_0, tr_1, X_ts, y_ts = data_splicing(credit_data.iloc[one_i], risk_labels.iloc[one_i], data_cols, 'risk_flag')    
    rfc = RandomForestClassifier(class_weight='balanced')
    X_tr = np.concatenate((tr_0[:, :tr_0.shape[1]-1], tr_1[:, :tr_1.shape[1]-1]), axis=0)
    y_tr = np.concatenate((tr_0[:, tr_0.shape[1]-1:], tr_1[:, tr_1.shape[1]-1:]), axis=0)
    rf_models = rfc.fit(X_tr, y_tr.ravel())
    predicted_flags = rfc.predict(X_ts)
#     predicted_flags = rf_team_predictions(rf_models, X_ts)
    print(classification_report(y_ts.ravel(), predicted_flags))
    print(accuracy_score(y_ts.ravel(), predicted_flags))
    print(np.unique(predicted_flags))

    print(confusion_matrix(y_ts.ravel(), predicted_flags))

    predicted_flags = rf_team_predictions(rf_models, credit_data.iloc[one_i].values)
    rf_model_team_preds.append(predicted_flags)
        
    return rf_model_team_preds

In [21]:
rf_model_team_preds = final_model_training_and_preds(credit_data, risk_labels, zero_i, one_i)

Train 0, 1: (146984, 6) (28996, 6)
[[0.60344828 1.         0.         0.         0.0074874 ]
 [0.10344828 0.         0.         0.         0.02739099]
 [0.13793103 0.         0.         0.         0.04210722]
 ...
 [0.86206897 0.         0.         1.         0.08915702]
 [0.4137931  1.         1.         1.         0.00185761]
 [0.48275862 0.         1.         1.         0.42203054]]
(175980,)
[[0.05172414 1.         0.         0.         0.01989574]
 [0.79310345 0.         1.         1.         0.00488109]
 [0.17241379 0.         1.         1.         0.00472912]
 ...
 [0.94827586 1.         0.         1.         0.01237156]
 [0.         0.         1.         1.         0.08580729]
 [0.74137931 0.         1.         1.         0.05155481]]
(4000,)
Accuracy: 0.97375
Model 1 --> Done
Accuracy: 0.977
Model 2 --> Done
Accuracy: 0.96975
Model 3 --> Done
              precision    recall  f1-score   support

         0.0       0.98      0.97      0.97      2000
         1.0       0.97    

In [22]:
credit_data.loc[zero_i, 'risk_flag'] = rf_model_team_preds[0]
credit_data.loc[one_i, 'risk_flag'] = rf_model_team_preds[1]

In [23]:
from sklearn.metrics import roc_auc_score

In [24]:
print(classification_report(risk_labels.values.ravel(), credit_data['risk_flag'].values.ravel()))
print(confusion_matrix(risk_labels.values.ravel(), credit_data['risk_flag'].values.ravel()))

              precision    recall  f1-score   support

           0       0.97      0.94      0.95    180389
           1       0.70      0.81      0.75     30996

    accuracy                           0.92    211385
   macro avg       0.83      0.88      0.85    211385
weighted avg       0.93      0.92      0.92    211385

[[169383  11006]
 [  5856  25140]]
